In [1]:
!pip install numba
!pip install scipy
!pip install dask
!pip install simpy
!pip install streamlit
!pip install fastapi
!pip install uvicorn

   ---------------------------------------- 0.0/94.6 kB ? eta -:--:--
   ----------------- ---------------------- 41.0/94.6 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 94.6/94.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/71.5 kB ? eta -:--:--
   ---------------------------------------- 71.5/71.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.8 kB ? eta -:--:--
   ------ --------------------------------- 10.2/62.8 kB ? eta -:--:--
   -------------------------- ------------- 41.0/62.8 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 62.8/62.8 kB 1.1 MB/s eta 0:00:00


In [13]:
import simpy
import numpy as np
from scipy.stats import norm

# Input parameters
demand_mean_monthly = 1000  # Mean monthly demand (units)
demand_std_monthly = 150    # Standard deviation of monthly demand (units)
monthly_max_demand = 1300   # Maximum monthly demand (units)
unit_cost = 50              # Cost per unit
holding_cost_monthly = 2     # Monthly holding cost per unit
monthly_plant_production_capacity = 1200  # Plant production capacity (units per month)
avg_lead_time_in_months = 1  # Average lead time in months
lead_time_std_in_months = 0.25  # Standard deviation of lead time in months
customer_order_quantity_monthly_avg = 950  # Avg monthly customer order quantity (units)
customer_order_quantity_monthly_std = 50  # Std of monthly customer order quantity (units)
customer_order_quantity_monthly_max = 1400  # Max monthly customer order quantity (units)
avg_on_hand_inventory_monthly = 500  # Avg on-hand inventory (units)
avg_in_transit_inventory_monthly = 200  # Avg in-transit inventory (units)
avg_oos_qty = 50  # Avg out-of-stock quantity
on_hand_inventory_monthly_std = 100  # Std of on-hand inventory (units)
in_transit_inventory_monthly_std = 50  # Std of in-transit inventory (units)
oos_qty_std = 10  # Std of out-of-stock quantity (units)
replenishment_days = 30  # Days between replenishment
service_level = 0.98  # Desired service level (98%)
days_in_month = 30  # Days in a month

# Safety Stock Calculation
def calculate_safety_stock():
    lead_time_demand_mean = avg_lead_time_in_months * demand_mean_monthly
    lead_time_demand_std = np.sqrt(avg_lead_time_in_months) * demand_std_monthly
    z_score_service_level = norm.ppf(service_level)
    safety_stock = z_score_service_level * lead_time_demand_std
    return safety_stock

# SimPy environment setup
class InventorySimulation:
    def __init__(self, env, ufr, safety_stock):
        self.env = env
        self.on_hand_inventory = avg_on_hand_inventory_monthly
        self.in_transit_inventory = avg_in_transit_inventory_monthly
        self.ufr = ufr
        self.safety_stock = safety_stock
        self.holding_cost = 0
        self.total_orders = 0
        self.orders_fulfilled = 0
        self.process = env.process(self.run())

    def run(self):
        while True:
            # Simulate demand for the month
            demand = np.random.normal(customer_order_quantity_monthly_avg, customer_order_quantity_monthly_std)
            demand = np.clip(demand, 0, customer_order_quantity_monthly_max)
            self.total_orders += demand

            # Check if current inventory can fulfill demand
            available_inventory = self.on_hand_inventory + self.in_transit_inventory
            if available_inventory >= self.ufr * demand:
                fulfilled = self.ufr * demand
                self.orders_fulfilled += fulfilled
                self.on_hand_inventory -= fulfilled
            else:
                # If not enough inventory, fulfill only available inventory
                fulfilled = available_inventory
                self.orders_fulfilled += fulfilled
                self.on_hand_inventory = 0

            # Calculate holding costs
            self.holding_cost += holding_cost_monthly * self.on_hand_inventory

            # Replenish inventory based on the replenishment cycle
            yield self.env.timeout(replenishment_days)
            replenishment_quantity = np.random.normal(monthly_plant_production_capacity, in_transit_inventory_monthly_std)
            self.in_transit_inventory = replenishment_quantity

            # Update on-hand inventory with in-transit stock
            self.on_hand_inventory += self.in_transit_inventory
            self.in_transit_inventory = 0

            # Ensure inventory meets safety stock requirement
            if self.on_hand_inventory < self.safety_stock:
                self.on_hand_inventory = self.safety_stock

# Simulation function for multiple UFR levels
def simulate_ufr_levels(ufr_levels):
    safety_stock = calculate_safety_stock()
    results = []

    for ufr in ufr_levels:
        env = simpy.Environment()  # Create a new environment for each simulation run
        simulation = InventorySimulation(env, ufr, safety_stock)
        env.run(until=365)  # Simulate for 1 year
        total_orders = simulation.total_orders
        orders_fulfilled = simulation.orders_fulfilled
        holding_cost = simulation.holding_cost
        ufr_actual = orders_fulfilled / total_orders if total_orders > 0 else 0

        results.append({
            "UFR Set": ufr,
            "Actual UFR": ufr_actual,
            "Holding Cost": holding_cost,
            "Total Orders": total_orders,
            "Orders Fulfilled": orders_fulfilled
        })

    return results

# Running the simulation for different UFR levels
def run_simulation():
    ufr_levels = [.75, .80, .85, .90, .95]  # Simulate UFR levels
    
    # Collect the simulation results
    results = simulate_ufr_levels(ufr_levels)

    # Display the results
    print("UFR Simulation Results:")
    print(f"{'UFR Set':<10}{'Actual UFR':<15}{'Holding Cost':<15}{'Total Orders':<15}{'Orders Fulfilled':<20}")
    for result in results:
        print(f"{result['UFR Set']:<10.2f}{result['Actual UFR']:<15.2f}{result['Holding Cost']:<15.2f}{result['Total Orders']:<15.2f}{result['Orders Fulfilled']:<20.2f}")

# Run the simulation
run_simulation()


UFR Simulation Results:
UFR Set   Actual UFR     Holding Cost   Total Orders   Orders Fulfilled    
0.75      0.75           73269.45       12206.06       9154.55             
0.80      0.80           67824.53       12321.35       9795.75             
0.85      0.85           61227.11       12335.68       10450.52            
0.90      0.89           50097.30       12540.46       11158.28            
0.95      0.94           46361.89       12368.14       11575.00            
